# Как работать с MiniO вручную

In [1]:
from minio import Minio
import os
from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv())

access_key = os.environ.get("MINIO_ACCESS_KEY")
secret_key = os.environ.get("MINIO_SECRET_KEY")

client = Minio(
    "data.pyrogn.ru",
    access_key=access_key,
    secret_key=secret_key,
    secure=True,
)

BUCKET_NAME = "tasksai"


In [2]:
[obj.object_name for obj in client.list_objects("tasksai")]

['.gitkeep',
 'tickets_src.csv',
 'custom_data/',
 'dataset_tickets_timeseries/',
 'nlp/',
 'tickets_daily/']